# 語意搜尋實作 (Semantic Search Implementation)

本筆記本展示如何使用 Sentence Transformers 和 FAISS 實現高效的語意搜尋。

## 實作步驟與說明

1. **環境設置**
   - 檢查 GPU 可用性
   - 設置計算裝置

2. **模型準備**
   - 載入預訓練的 Sentence Transformer 模型
   - 使用輕量級但高效的 all-MiniLM-L6-v2 模型

3. **數據處理**
   - 載入 MS MARCO 問答數據集
   - 提取文檔和查詢數據

4. **向量索引**
   - 使用 FAISS 建立高效的向量索引
   - 支援 GPU 加速搜尋

5. **搜尋評估**
   - 執行批次搜尋
   - 計算處理時間
   - 分析搜尋結果

In [1]:
# 環境設置：檢查 GPU 可用性並設置計算裝置
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'使用的計算裝置: {device}')

使用的計算裝置: cuda


In [2]:
# 載入預訓練模型
# all-MiniLM-L6-v2 是一個輕量級模型，在效能和速度上取得良好平衡
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device=device)
print('模型載入完成')
model

模型載入完成


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [3]:
# 載入並準備數據
# MS MARCO 是微軟發布的大規模機器閱讀理解和問答數據集
from datasets import load_dataset

print('開始載入數據集...')
dataset_name = 'ms_marco'
dataset = load_dataset(dataset_name, 'v2.1', split='train').select(range(100000))

# 整理文檔和查詢數據
documents = [
    passage_text
    for passages in dataset['passages']
    for passage_text in passages['passage_text']
]

queries = dataset['query']

# 隨機選擇查詢樣本進行測試
NUM_QUERIES = 10000
import random
random.shuffle(queries)
queries = queries[:NUM_QUERIES]

print('數據準備完成')
print(f'\n示例文檔:\n{documents[0]}')
print(f'\n示例查詢:\n{queries[0]}')

開始載入數據集...
數據準備完成

示例文檔:
The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.

示例查詢:
abeyance legal definition ma


## FAISS 向量索引與搜尋

FAISS (Facebook AI Similarity Search) 是一個高效能的向量索引和搜尋函式庫：

- 支援十億級別的向量搜尋
- 提供 GPU 加速功能
- 實現多種索引算法
- 優化的記憶體使用

In [4]:
import faiss

print('開始建立向量索引...')
# 將文檔轉換為向量表示
fs_documents_embedding = model.encode(documents, batch_size=256, convert_to_tensor=True)
fs_documents_embedding_np = fs_documents_embedding.cpu().numpy()

# 創建 FAISS 索引 (使用 L2 歐氏距離)
dimension = fs_documents_embedding.shape[1]
index = faiss.IndexFlatL2(dimension)

# GPU 加速處理
if torch.cuda.is_available():
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
    index = gpu_index

# 添加文檔向量到索引
index.add(fs_documents_embedding_np)
print(f'向量維度: {dimension}')
print('索引建立完成')

開始建立向量索引...
向量維度: 384
索引建立完成


In [5]:
import time
import util

print('開始執行搜尋...')
# 將查詢轉換為向量
fs_queries_embedding = model.encode(queries, batch_size=256, convert_to_tensor=True)
fs_queries_embedding_np = fs_queries_embedding.cpu().numpy()

# 設定每個查詢返回的結果數量
top_k = 2

# 執行批次搜尋並計時
start_time = time.time()
D, I = util.batch_search(fs_queries_embedding_np, index, top_k, batch_size=8)
end_time = time.time()

# 顯示部分搜尋結果
print('\n搜尋結果示例：')
for i, (dists, idxs) in enumerate(zip(D, I), 1):
    print(f'\n查詢 {i}: {queries[i-1]}')
    for j, (dist, idx) in enumerate(zip(dists, idxs), 1):
        print(f'   結果 {j}')
        print(f'\t距離: {dist:.4f}')  # 距離越小表示越相似
        print(f'\t文檔: {documents[idx]}')

    # 只顯示前 5 個查詢的結果
    if i == 5:
        break

開始執行搜尋...

搜尋結果示例：

查詢 1: abeyance legal definition ma
   結果 1
	距離: 0.4080
	文檔: Freebase(0.00 / 0 votes)Rate this definition: Abeyance is a state of expectancy in respect of property, titles or office, when the right to them is not vested in any one person, but awaits the appearance or determination of the true owner.
   結果 2
	距離: 0.4314
	文檔: Legal Definition of abeyance. 1  1 : a lapse in the succession of property during which there is no person in whom title to the property is vested —usually used with in the estate was in abeyance.

查詢 2: how to listen to offline mode on pandora
   結果 1
	距離: 0.3602
	文檔: To avoid using your mobile data, you have the option to manually enable the Offline Listening option. 1  Go to your Station List or the Pandora Home Screen. 2  Tap the Menu icon > Offline Mode. 3  Here you can toggle the Offline Listening feature ON (green).
   結果 2
	距離: 0.4026
	文檔: Back To Top. Turn Offline Listening On. To avoid using your mobile data, you have the option to manua

## 效能分析

計算並顯示搜尋效能指標：
- 總處理時間
- 平均每次查詢時間
- GPU 加速效果

In [6]:
# 計算效能指標
total_time = end_time - start_time
avg_time_per_query = total_time / NUM_QUERIES

print(f'搜尋效能統計：')
print(f'總處理時間: {total_time:.2f} 秒')
print(f'平均每次查詢時間: {avg_time_per_query*1000:.2f} 毫秒')
print(f'查詢總數: {NUM_QUERIES}')
print(f'批次大小: 8')
print(f'使用設備: {"GPU" if torch.cuda.is_available() else "CPU"}')
print(f'向量維度: {dimension}')

搜尋效能統計：
總處理時間: 14.00 秒
平均每次查詢時間: 1.40 毫秒
查詢總數: 10000
批次大小: 8
使用設備: GPU
向量維度: 384
